In [38]:
!pip install torchsummary
from torchsummary import summary
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [64]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # CLASStorch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        #  conv2d format -> input channels, output channels, kernel size, kwargs
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF  
        # conv1 : 
        # input -> 1 x 28 x 28
        # output -> 32 x 28 x 28
        # receptive field -> 3 x 3 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # conv2 : 
        # input -> 32 x 28 x 28
        # output -> 64 x 28 x 28
        # receptive field -> 5 x 5
        self.pool1 = nn.MaxPool2d(2, 2)
        # max pooled : 
        # input -> 64 x 28 x 28
        # output -> 64 x 14 x 14
        # receptive field -> 10 x 10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # conv3 : 
        # input -> 64 x 14 x 14
        # output -> 128 x 14 x 14
        # receptive field -> 12 x 12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
         # conv1 : 
        # input -> 128 x 14 x 14
        # output -> 256 x 14 x 14
        # receptive field -> 14 x 14
        self.pool2 = nn.MaxPool2d(2, 2)
        # max pooled : 
        # input -> 256 x 14 x 14
        # output -> 256 x 7 x 7
        # receptive field -> 28 x 28
        self.conv5 = nn.Conv2d(256, 512, 3)
        # con5 : 
        # input -> 256 x 7 x 7
        # output -> 512 x 5 x 5
        # receptive field -> 30 x 30
        self.conv6 = nn.Conv2d(512, 1024, 3)
        # conv6 : 
        # input -> 512 x 5 x 5
        # output -> 1024 x 3 x 3
        # receptive field -> 32 x 32
        self.conv7 = nn.Conv2d(1024, 10, 3)
        # conv7 : 
        # input -> 1024 x 3 x 3
        # output -> 10 x 1 x 1
        # receptive field -> 34 x 34
    def forward(self, x):
        x = self.pool1(self.conv2(self.conv1(x)))
        x = self.pool2(self.conv4(self.conv3(x)))
        x = self.conv6(self.conv5(x))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=0)

In [65]:
model = Net()
print(model.conv1.in_channels, model.conv1.out_channels)

1 32


In [66]:
# checking if we have cuda compatible GPU
# if not, we'll use CPU instead 😪
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
# spills out summary of the model based on a given input size
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

In [67]:
# to pop out similar randomness (deterministic) 
torch.manual_seed(1)
batch_size = 128

# to keep data in memory, pinned memory helps in faster data transfer 
# to gpu
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [68]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [74]:
model = Net().to(device)
# stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=2.5393459796905518 batch_id=468: 100%|██████████| 469/469 [00:35<00:00, 13.26it/s]



Test set: Average loss: 2.8411, Accuracy: 9767/10000 (98%)

